<h3>Team Forest<br/>
Biomedial Image Processing Application (BIPA)<br/>
CSC821, Prof. K. Okada<br/>
Spring 2022</h3>

Please note: this code uses the Dicom images downloadable at https://wiki.cancerimagingarchive.net/pages/viewpage.action?pageId=80969742#80969742bcab02c187174a288dbcbf95d26179e8. Downloading the images requires first installing the NBIA Data Retriever on your local machine which is available in the App Store.

<b>Image I/O</b>

In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
import SimpleITK as sitk

import sys
import os

In [24]:
# example of reading Dicom images from COVID dataset, exporting, and showing outputting 3D image

# TO-DO: manually setting path here, need an interface to take input
dicom_dir = "NBIA_images/manifest-1608266677008/MIDRC-RICORD-1A/MIDRC-RICORD-1A-419639-000082/08-02-2002-NA-CT CHEST WITHOUT CONTRAST-04614/2.000000-ROUTINE CHEST NON-CON-97100"

print("Reading Dicom directory:", dicom_dir)
reader = sitk.ImageSeriesReader()

dicom_names = reader.GetGDCMSeriesFileNames(dicom_dir)

reader.SetFileNames(dicom_names)

image = reader.Execute()

size = image.GetSize()
print("Image size:", size[0], size[1], size[2])

print("Writing image:", "test_image.nii")

sitk.WriteImage(image, "test_image.nii")

if ("SITK_NOSHOW" not in os.environ):
    sitk.Show(image, "Dicom Series")

Reading Dicom directory: NBIA_images/manifest-1608266677008/MIDRC-RICORD-1A/MIDRC-RICORD-1A-419639-000082/08-02-2002-NA-CT CHEST WITHOUT CONTRAST-04614/2.000000-ROUTINE CHEST NON-CON-97100
Image size: 512 512 255
Writing image: test_image.nii


In [37]:
# example of 3D image attributes
covid_example = sitk.ReadImage("./test_image.nii")

print(f"origin: {covid_example.GetOrigin()}")
print(f"size: {covid_example.GetSize()}")
print(f"spacing: {covid_example.GetSpacing()}")
print(f"direction: {covid_example.GetDirection()}\n")

print("Dimensions: " + str(covid_example.GetDimension()))
print("Width: " + str(covid_example.GetWidth()))
print("Height: " + str(covid_example.GetHeight()))
print("Depth: " + str(covid_example.GetDepth()))
print("Pixel ID: " + str(covid_example.GetPixelIDValue()))
print("Pixel Type: " + str(covid_example.GetPixelIDTypeAsString()))
print("Components Per Pixel: " + str(covid_example.GetNumberOfComponentsPerPixel()))

origin: (-192.1999969482422, -180.0, -349.25)
size: (512, 512, 255)
spacing: (0.703125, 0.703125, 1.25)
direction: (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)

Dimensions: 3
Width: 512
Height: 512
Depth: 255
Pixel ID: 4
Pixel Type: 32-bit signed integer
Components Per Pixel: 1


In [43]:
# extract modalities of various embedded scans
from ipywidgets import interact, fixed

data_directory = "NBIA_images/manifest-1608266677008/MIDRC-RICORD-1A/MIDRC-RICORD-1A-419639-000361/10-21-2002-NA-CT CHEST WITHOUT CONTRAST-91670/2.000000-ROUTINE CHEST NON-CON-50599"
# Global variable 'selected_series' is updated by the interact function
selected_series = ""
file_reader = sitk.ImageFileReader()


def DICOM_series_dropdown_callback(series_to_load, series_dictionary):
    global selected_series
    # Print some information about the series from the meta-data dictionary
    # DICOM standard part 6, Data Dictionary: http://medical.nema.org/medical/dicom/current/output/pdf/part06.pdf
    file_reader.SetFileName(series_dictionary[series_to_load][0])
    file_reader.ReadImageInformation()
    tags_to_print = {
        "0010|0010": "Patient name: ",
        "0008|0060": "Modality: ",
        "0008|0021": "Series date: ",
        "0008|0080": "Institution name: ",
        "0008|1050": "Performing physician's name: ",
    }
    for tag in tags_to_print:
        try:
            print(tags_to_print[tag] + file_reader.GetMetaData(tag))
        except:  # Ignore if the tag isn't in the dictionary
            pass
    selected_series = series_to_load


# Directory contains multiple DICOM studies/series, store
# in dictionary with key being the series ID
reader = sitk.ImageSeriesReader()
series_file_names = {}
series_IDs = reader.GetGDCMSeriesIDs(data_directory)
# Check that we have at least one series
if series_IDs:
    for series in series_IDs:
        series_file_names[series] = reader.GetGDCMSeriesFileNames(
            data_directory, series
        )

    interact(
        DICOM_series_dropdown_callback,
        series_to_load=list(series_IDs),
        series_dictionary=fixed(series_file_names),
    )
else:
    print("Data directory does not contain any DICOM series.")

interactive(children=(Dropdown(description='series_to_load', options=('1.2.826.0.1.3680043.10.474.419639.44260…

<b>Registration</b>

<b>Segmentation</b>

<b>Quantification</b>